# Klasyfikacja urządzenia na podstawie cech sygnału elektrycznego
W tym notatniku znajduje się prosta implementacja klasyfikacji dla jednego urządzenia na podstawie cech sygnałów prądu, napięcia i mocy. Wykorzystane proste cechy statystyczne oraz klasyfikatory: KNN, Random Forest, SVM i sieć neuronową (MLP).

In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

## Wczytanie danych pomiarowych dla komputera

In [ ]:
df = pd.read_csv('komputer.csv')
df.head()

## Ekstrakcja prostych cech z sygnału
Wyliczamy cechy z okien czasowych (np. co 20 próbek), takie jak:
- średnia i odchylenie mocy,
- maksymalny prąd,
- minimalne napięcie,
- średni prąd,
- odchylenie napięcia.

In [ ]:
window_size = 20

samples = []

for i in range(0, len(df) - window_size, window_size):
    window = df.iloc[i:i+window_size]
    
    features = {
        'mean_watt': window['watt'].mean(),
        'std_watt': window['watt'].std(),
        'max_current': window['current'].max(),
        'min_voltage': window['voltage'].min(),
        'mean_current': window['current'].mean(),
        'std_voltage': window['voltage'].std(),
        'label': 1  # komputer
    }
    samples.append(features)

data = pd.DataFrame(samples)
data.head()

# Oznaczenie klasy 0 (nieaktywne) i 1 (aktywne) na podstawie wartości mocy
Załóżmy, że próg 5 W oddziela aktywność od braku działania

In [6]:
data['label'] = (data['watt'] > 5).astype(int)

# Sprawdźmy rozkład klas
data['label'].value_counts()
full_data = data.copy()

NameError: name 'data' is not defined

## Przygotowanie danych do klasyfikacji
Normalizacja cech i podział danych na zbiory treningowe i testowe.

In [ ]:
X = full_data.drop('label', axis=1)
y = full_data['label']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.3, random_state=42
)

## Trening i ocena klasyfikatorów
Trenujemy i testujemy 4 klasyfikatory: KNN, Random Forest, SVM oraz MLP (prosta sieć neuronowa). Wypisujemy dokładność predykcji.

In [ ]:
# KNN
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)
y_pred_knn = knn.predict(X_test)
print("KNN Accuracy:", accuracy_score(y_test, y_pred_knn))

# Random Forest
rf = RandomForestClassifier(n_estimators=100)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
print("Random Forest Accuracy:", accuracy_score(y_test, y_pred_rf))

# SVM
svm = SVC(kernel='rbf')
svm.fit(X_train, y_train)
y_pred_svm = svm.predict(X_test)
print("SVM Accuracy:", accuracy_score(y_test, y_pred_svm))

# MLP (Neural Network)
mlp = MLPClassifier(hidden_layer_sizes=(64, 32), max_iter=500)
mlp.fit(X_train, y_train)
y_pred_mlp = mlp.predict(X_test)
print("MLP Accuracy:", accuracy_score(y_test, y_pred_mlp))

## Macierze pomyłek (Confusion Matrices)
Wizualizacja tego, jak dobrze klasyfikatory rozróżniają klasę "lodówka" i "brak urządzenia".

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

def show_conf_matrix(y_true, y_pred, title):
    cm = confusion_matrix(y_true, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Brak", "Lodówka"])
    disp.plot(cmap="Blues", values_format='d')
    plt.title(title)
    plt.show()

show_conf_matrix(y_test, y_pred_knn, "KNN")
show_conf_matrix(y_test, y_pred_rf, "Random Forest")
show_conf_matrix(y_test, y_pred_svm, "SVM")
show_conf_matrix(y_test, y_pred_mlp, "MLP")

## Metryki klasyfikacji (Precision, Recall, F1-score)
Zestawienie metryk, które pokazują skuteczność detekcji każdej z klas.

In [ ]:
from sklearn.metrics import classification_report

print("KNN:\n", classification_report(y_test, y_pred_knn, target_names=["Brak", "Lodówka"]))
print("Random Forest:\n", classification_report(y_test, y_pred_rf, target_names=["Brak", "Lodówka"]))
print("SVM:\n", classification_report(y_test, y_pred_svm, target_names=["Brak", "Lodówka"]))
print("MLP:\n", classification_report(y_test, y_pred_mlp, target_names=["Brak", "Lodówka"]))

## Sprawdzenie liczby próbek w każdej klasie
Upewniamy się, że klasy są zrównoważone.

In [ ]:
full_data['label'].value_counts()

## Wizualizacja PCA – podgląd separacji klas
Redukcja wymiarów do 2D dla szybkiego podglądu, jak rozkładają się klasy.

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

plt.figure(figsize=(8, 6))
plt.scatter(X_pca[:, 0], X_pca[:, 1], c=y, cmap='coolwarm', alpha=0.6)
plt.title("Rozkład klas po redukcji PCA (2D)")
plt.xlabel("PCA 1")
plt.ylabel("PCA 2")
plt.grid(True)
plt.show()